In [1]:
import os
import timeit

import cv2
from tensorflow.keras.backend import clear_session
import numpy as np
from PIL import Image
from scipy.stats import entropy, norm

import image_process as ip
import tools as tl
import tensorflow as tf
from tensorflow.keras.applications import Xception
import gc
import models
from tensorflow.keras.optimizers import Adam
from keras_bcnn.models import BayesianUNet2D
from matplotlib import pylab as plt
from scipy.stats import entropy
from tensorflow.keras.utils import plot_model
from keras_bcnn.models import MCSampler
import scipy.misc

Using TensorFlow backend.


In [2]:
data='ips'
mode='test'
data_num=4
method='unet_patch'
patch_size=160
n_class=3
reflect=False

In [3]:
for mode in ["test"] : 
    for data_num in [1,2,3,4,5]:
        save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
        os.makedirs(save_path + 'stage2/%s_un/uncertainty'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage2/%s_un/uncertainty_T_0.5'%mode, exist_ok=True)
        os.makedirs(save_path+"stage2/%s"%mode, exist_ok=True)
        os.makedirs(save_path+"stage2/label", exist_ok=True)
        os.makedirs(save_path + 'stage2/%s_un/correctness'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage2/%s_un/probability'%mode, exist_ok=True)

In [4]:
for mode in ["test","train","val"] : 
    for data_num in [1,2,3,4,5]:
        save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
        save_path = save_path+"stage2/"
        os.makedirs(save_path + 'stage3/%s_un/uncertainty'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage3/%s_un/uncertainty_T_0.5'%mode, exist_ok=True)
        os.makedirs(save_path+"stage3/%s"%mode, exist_ok=True)
        os.makedirs(save_path+"stage3/label", exist_ok=True)
        os.makedirs(save_path + 'stage3/%s_un/correctness'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage3/%s_un/probability'%mode, exist_ok=True)

In [4]:
for mode in ["test","train","val"] : 
    for data_num in [1,2,3,4,5]:
        save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
        save_path = save_path+"stage2/stage3/"
        os.makedirs(save_path + 'stage4/%s_un/uncertainty'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage4/%s_un/uncertainty_T_0.5'%mode, exist_ok=True)
        os.makedirs(save_path+"stage4/%s"%mode, exist_ok=True)
        os.makedirs(save_path+"stage4/label", exist_ok=True)
        os.makedirs(save_path + 'stage4/%s_un/correctness'%mode, exist_ok=True)
        os.makedirs(save_path + 'stage4/%s_un/probability'%mode, exist_ok=True)

In [12]:
for mode in ["test","train"] :
    for data_num in [1]:
        save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
        os.makedirs(save_path + '%s_un/uncertainty'%mode, exist_ok=True)
        os.makedirs(save_path + '%s_un/uncertainty_T_0.5'%mode, exist_ok=True)
        os.makedirs(save_path+"%s"%mode, exist_ok=True)
        os.makedirs(save_path+"label", exist_ok=True)
        os.makedirs(save_path + '%s_un/correctness'%mode, exist_ok=True)
        os.makedirs(save_path + '%s_un/probability'%mode, exist_ok=True)

In [4]:
def get_ytrue(mask_array, data):
        if data == "ips":
            height, width, _ = mask_array.shape
            good_label = ((mask_array[:,:,0] == 255)&
                          (mask_array[:,:,1] == 0)&
                          (mask_array[:,:,2] == 0)
                         ) * np.ones((height, width)) * 1
            bad_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 255)&
                         (mask_array[:,:,2] == 0)
                        ) * np.ones((height, width)) * 2
            bgd_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 0)&
                         (mask_array[:,:,2] == 255)
                        ) * np.ones((height, width)) * 3
            y_true = good_label + bad_label + bgd_label
            return y_true
        else:
            height, width = mask_array.shape
            y_true = (mask_array == 255) * np.ones((height, width))
            return y_true

In [5]:
def make_prediction_map(size, test_step, width, height, pre_map, ref_num, reflect, data, n_class):
    print("Start visualize image...")
    num = 0
    if reflect:
        height += ref_num*2
        width += ref_num*2
    pre_label = np.zeros((height,width,n_class))
    for y in range(0,height-size+1,test_step):
        for x in range(0,width-size+1,test_step):
            patch = np.ones((size,size,n_class))
            patch = patch*pre_map[num]
            pre_label[y:y+size,x:x+size,:] += patch
            num += 1
    sumaltion=np.sum(pre_label,axis=2)
    pre_label[:,:,0]=pre_label[:,:,0]/sumaltion
    pre_label[:,:,1]=pre_label[:,:,1]/sumaltion
    pre_label[:,:,2]=pre_label[:,:,2]/sumaltion
    e=entropy(pre_label,axis=2)
    max=pre_label.max(axis=2)
    pro = np.zeros((height,width,n_class))
    pro1=pre_label[:,:,0]
    pro2=pre_label[:,:,1]
    pro3=pre_label[:,:,2]
    pro1[pro1<max]=0
    pro2[pro2<max]=0
    pro3[pro3<max]=0
    pro[:,:,0]=pro1
    pro[:,:,1]=pro2
    pro[:,:,2]=pro3
    pro*=255   
    if data in ['ips']:
        vis_img, label_img = tl.get_InfImg(pre_label, data)
    elif data in ['melanoma', 'colonoscopy_tissue']:
        others = pre_label[:,:,1]
        target = pre_label[:,:,0]
        label_img = (target > others)*1
        vis_img = label_img*255
    if reflect:
        vis_img = vis_img[ref_num:height-ref_num,ref_num:width-ref_num]
        label_img = label_img[ref_num:height-ref_num,ref_num:width-ref_num]
    return vis_img, label_img,e,pro

In [6]:
def convert2label(name, save_path):
    mask = np.array(Image.open(save_path+"/test/" + name + '.png'), dtype=int)
    label = mask / 255
    if data == 'ips':
        label = label[:, :, 0] * 1 + label[:, :, 1] * 2 + label[:, :, 2] * 3
        label = Image.fromarray(np.uint8(label))
        label.save(save_path + '/label/' + name + '.png')
    else:
        label = Image.fromarray(np.uint8(label))
        label.save(save_path + '/label/' + name + '.png')

In [11]:
#prediction, label map, uncertainty mapの作成
s=0.5
ite=1
test_slide_step=50

for data_num in [1]:
    clear_session()
    save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
    save_path = save_path#+'stage2/'#stage3/stage4/'#################################
    model = BayesianUNet2D((patch_size, patch_size, 3), 3).build()
    model.load_weights(save_path+'weights/weights.h5')
    for mode in ["test","train"]:
        img_path, mask_path = tl.data_path_load(data,mode,data_num)
        ImageLoader = ip.PatchImageLoader(img_path,patch_size,test_slide_step,data,mode)
        for i_path in img_path:
            print("************************************")
            print(i_path)
            name = os.path.basename(i_path)[:-4]
            X_test, width, height = ImageLoader.crop_one_img(i_path, patch_size, reflect)
            X_test = np.array(X_test) / 255.
            Yp=0
            for i in range(ite):
                Yp += model.predict(X_test)
            Yp/=ite
            del X_test
            gc.collect()
            print("Predict finish")
            vis_img, label_img,e,pro = make_prediction_map(patch_size,
                                                             test_slide_step,
                                                             width,
                                                             height,
                                                             Yp,
                                                             patch_size,
                                                             reflect,
                                                             data,
                                                             n_class)
            vis_img = vis_img[:, :, [2, 1, 0]]
            pro = pro[:, :, [2, 1, 0]]
            cv2.imwrite(save_path + '%s/' % mode+name + '.png', vis_img)
            cv2.imwrite(save_path + '%s_un/probability/' % mode+name + '.png', pro)
            if mode == "test":
                convert2label(name, save_path)
            tmp=e
            e_1 = e*255
            e_1 = Image.fromarray(e_1)
            e_1 = e_1.convert('RGB')
            e_1.save(save_path + '%s_un/uncertainty/'%mode + name + '.png')
            e[e>=s]=255####しきい値設定
            e[e<s]=0
            e=Image.fromarray(e)
            e = e.convert('RGB')
            e.save(save_path + '%s_un/uncertainty_T_0.5/'%mode + name + '.png')
            #tmp[tmp>=0.7]=255
            #tmp[tmp<0.7]=0
            #tmp=Image.fromarray(tmp)
            #tmp = tmp.convert('RGB')
            #tmp.save(save_path + '/uncertainty_T_0.7/' + name + '.png')
            del Yp,vis_img, label_img,e,e_1
            gc.collect()
            print("************************************")
    del model
    gc.collect()

************************************
data/ips/img/E3230654.JPG
Predict finish
Start visualize image...


/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in greater_equal
/home/sora/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in less


************************************
************************************
data/ips/img/E3230660.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230680.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230684.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230686.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230687.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230694.JPG
Predict finish
Start visualize image...
************************************
************************************
data/ips/img/E3230710.JPG
Predict finish
Start visualize image...
********************

FileNotFoundError: [Errno 2] No such file or directory: './ips/unet_patch/unet/size_160/dataset_1/train_un/uncertainty/E3230652.png'

In [13]:
def pre_data_path_load(path):
    """
    data: str, "ips" or "melanoma"
    mode: str, "train" or "test"
    d_num: int, 1, ..., 5
    """
    img_path=[]
    if data in ['ips', 'melanoma']:
        for x in os.listdir(path):
            img_path.append(path+x)
        return img_path

In [14]:
#correctness mapの作成
for data_num in [1]:
    img_path, mask_path = tl.data_path_load(data,'test',data_num)
    mode="test"
    model_name="unet"
    size=160
    path=("./%s/%s/%s/size_%d/dataset_%d/" %(data, method,model_name,size,data_num))
    #path=("./%s/%s/%s/size_%d/dataset_%d/stage2/" %(data, method,model_name,size,data_num))

    pre_path = pre_data_path_load(path+"%s/"%mode)

    mask_path.sort()
    pre_path.sort()
    for m_path,p_path in zip(mask_path,pre_path):
        print(m_path)
        name = os.path.basename(m_path)[:-4]
        m_mask = cv2.imread(m_path)
        m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
        height, width = m_mask.shape[:2]
        imim = m_mask[:,:,0]+m_mask[:,:,1]+m_mask[:,:,2]
        m_mask[:,:,0]=(imim==255)*m_mask[:,:,0]
        m_mask[:,:,1]=(imim==255)*m_mask[:,:,1]
        m_mask[:,:,2]=(imim==255)*m_mask[:,:,2]

        mask = get_ytrue(m_mask, 'ips')
        oor = ~(mask == 0) *1

        p_image = cv2.imread(p_path)
        p_image = cv2.cvtColor(p_image, cv2.COLOR_BGR2RGB)
        p_image[:,:,0] = p_image[:,:,0]* oor 
        p_image[:,:,1] = p_image[:,:,1] * oor 
        p_image[:,:,2] = p_image[:,:,2] * oor
        pre = get_ytrue(p_image, 'ips')
        true=mask-pre
        true[true!=0]=255
        true=true*oor
        true=Image.fromarray(true)
        true = true.convert('RGB')
        true.save(path + 'test_un/correctness/' + name + '.png')


data/ips/mask/E3230654.png
data/ips/mask/E3230660.png
data/ips/mask/E3230680.png
data/ips/mask/E3230684.png
data/ips/mask/E3230686.png
data/ips/mask/E3230687.png
data/ips/mask/E3230694.png
data/ips/mask/E3230710.png
data/ips/mask/E3230717.png
data/ips/mask/E3230721.png
data/ips/mask/E3230725.png
data/ips/mask/E3230726.png


In [6]:
def probability_load(data, mode, method,model_name,size,d_num):
    s1_path=[]
    s2_path=[]
    if data in ['ips', 'melanoma']:
        path=("./%s/%s/%s/size_%d/dataset_%d/stage2/%s_un/probability/" %(data, method,model_name,size,d_num,mode))
        for x in os.listdir("./%s/%s/%s/size_%d/dataset_%d/stage2/%s_un/probability/" %(data, method,model_name,size,d_num,mode)):
            s1_path.append(path+x)
            
        path=("./%s/%s/%s/size_%d/dataset_%d/%s_un/probability/" %(data, method,model_name,size,d_num,mode))
        for y in os.listdir("./%s/%s/%s/size_%d/dataset_%d/%s_un/probability/" %(data, method,model_name,size,d_num,mode)):
            s2_path.append(path+y)
        return s1_path,s2_path

In [3]:
def Comparison_load(data, mode, method,model_name,size,d_num):
    s1_path=[]
    s2_path=[]
    if data in ['ips', 'melanoma']:
        path=("./%s/%s/%s/size_%d/dataset_%d/stage2/%s/" %(data, method,model_name,size,d_num,mode))
        for x in os.listdir("./%s/%s/%s/size_%d/dataset_%d/stage2/%s/" %(data, method,model_name,size,d_num,mode)):
            s2_path.append(path+x)
            
        path=("./%s/%s/%s/size_%d/dataset_%d/%s/" %(data, method,model_name,size,d_num,mode))
        for y in os.listdir("./%s/%s/%s/size_%d/dataset_%d/%s/" %(data, method,model_name,size,d_num,mode)):
            s1_path.append(path+y)
        return s1_path,s2_path

In [16]:
#segmentationの改善を赤、悪化を青で表す
#多分使わない
for data_num in [3]: 
    save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
    img_path, mask_path = tl.data_path_load(data,'test',data_num)
    #pre_path = tl.pre_data_path_load(data,'test',method,"unet",160,data_num)
    pre1_path,pre2_path = Comparison_load(data,'test',method,"unet",160,data_num)
    os.makedirs(save_path + 'stage2/diff', exist_ok=True)
    mask_path.sort()
    pre1_path.sort()
    pre2_path.sort()
    print(len(pre1_path))
    print("dataset_%d"%data_num)
    for m_path,p_path,p2_path in zip(mask_path,pre1_path,pre2_path):
        name = os.path.basename(m_path)[:-4]
        m_mask = cv2.imread(m_path)
        m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
        height, width = m_mask.shape[:2]
        imim = m_mask[:,:,0]+m_mask[:,:,1]+m_mask[:,:,2]
        m_mask[:,:,0]=(imim==255)*m_mask[:,:,0]
        m_mask[:,:,1]=(imim==255)*m_mask[:,:,1]
        m_mask[:,:,2]=(imim==255)*m_mask[:,:,2]

        mask = get_ytrue(m_mask, 'ips')
        oor = ~(mask == 0) *1

        p_image = cv2.imread(p_path)
        p_image = cv2.cvtColor(p_image, cv2.COLOR_BGR2RGB)
        p_image[:,:,0] = p_image[:,:,0]* oor 
        p_image[:,:,1] = p_image[:,:,1] * oor 
        p_image[:,:,2] = p_image[:,:,2] * oor
    
        p2_image = cv2.imread(p2_path)
        p2_image = cv2.cvtColor(p2_image, cv2.COLOR_BGR2RGB)
        p2_image[:,:,0] = p2_image[:,:,0]* oor 
        p2_image[:,:,1] = p2_image[:,:,1] * oor 
        p2_image[:,:,2] = p2_image[:,:,2] * oor
        p1=p_image.astype(np.int16)
        p2=p2_image.astype(np.int16)
        m=m_mask.astype(np.int16)
        diff=p1-m#-p_image
        diff2=p2-m#p2_image
        #pre = get_ytrue(p_image, 'ips')
        #true=mask-pre
        diff[diff<0]=0
        diff2[diff2<0]=0
        diff = diff[:, :, 0] * 1 + diff[:, :, 1] * 2 + diff[:, :, 2] * 3
        diff2 = diff2[:, :, 0] * 1 + diff2[:, :, 1] * 2 + diff2[:, :, 2] * 3
        diff[diff>0]=255
        diff2[diff2>0]=255

        d=diff2-diff
        d2=diff-diff2
        d[d<0]=0
        d2[d2<0]=0
        comparison=np.zeros((height, width,3))
        comparison[:,:,0]=d
        comparison[:,:,2]=d2
        #comparison=Image.fromarray(comparison)
        #comparison = comparison.convert('RGB')
        #comparison.save(save_path + 'stage2/diff/' + name + '.png')
        cv2.imwrite(save_path + 'stage2/diff/' + name + '.png', comparison)

11
dataset_3
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230652.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230666.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230679.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230685.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230688.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230693.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230698.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230704.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230707.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230708.png
./ips/unet_patch/unet/size_160/dataset_3/stage2/test/E3230719.png


In [17]:
def diff_load(data, mode, method,model_name,size,d_num):
    dpath=[]
    if data in ['ips', 'melanoma']:
        path=("./%s/%s/%s/size_%d/dataset_%d/stage2/diff/" %(data, method,model_name,size,d_num))
        for x in os.listdir("./%s/%s/%s/size_%d/dataset_%d/stage2/diff/" %(data, method,model_name,size,d_num)):
            dpath.append(path+x)
            
        return dpath

In [18]:
#segmentationの改善を赤、悪化を青で表し、コロニー画像と合体させる。先に上２つのセルを実行
#多分使わない
mode = "test"
for data_num in [3]:
    print("%d"%data_num)
    save_path = tl.get_save_path(data, method, 'unet',patch_size, data_num)
    save_path = save_path + "stage2/"
    os.makedirs(save_path + '%s_un/difference'%mode, exist_ok=True)
    img_path, mask_path = tl.data_path_load(data,'test',data_num)
    diff_path = diff_load(data,'test',method,"unet",160,data_num)
    img_path.sort()
    mask_path.sort()
    diff_path.sort()
    for i_path,d_path,m_path in zip(img_path,diff_path,mask_path):
        name = os.path.basename(i_path)[:-4]
        img = Image.open(i_path)
        diff = Image.open(d_path)

        diff = diff.copy()
        diff.putalpha(128)
        
        m_mask = cv2.imread(m_path)
        m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
        height, width = m_mask.shape[:2]
        imim = m_mask[:,:,0]+m_mask[:,:,1]+m_mask[:,:,2]
        m_mask[:,:,0]=(imim==255)*m_mask[:,:,0]
        m_mask[:,:,1]=(imim==255)*m_mask[:,:,1]
        m_mask[:,:,2]=(imim==255)*m_mask[:,:,2]

        mask = get_ytrue(m_mask, 'ips')
        oor = ~(mask == 0) *1

        diff2 = Image.open(d_path)
        diff2 = np.array(diff2)
        diff2[:,:,0] = diff2[:,:,0] * oor
        diff2[:,:,1] = diff2[:,:,1] * oor
        diff2[:,:,2] = diff2[:,:,2] * oor
        white_pixels = (diff2 == (0, 0, 0)).all(axis=-1)
        diff2[white_pixels] = (255, 255, 255)
        diff2[diff2>0]=255        
        diff2=Image.fromarray(diff2).convert("L")

        mask = Image.new("L", diff.size, 128)
        im = Image.composite(img, diff, diff2)
        im.save(save_path + '%s_un/difference/'%mode + name + '.png')

3
